<a href="https://colab.research.google.com/github/Shagun20/CSE508_Winter2023_A1_112/blob/main/IR_Assn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Q1) Preprocessing 
  

PART (i)


In [ ]:
import os

folder_path = '/content/drive/MyDrive/IR_AssnDatasets_Sem6/Assn1/CSE508_Winter2023_Dataset'


for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path) and file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            file_content = f.read()
            print(file_content)

In [ ]:
import os

docs= []
folder_path = '/content/drive/MyDrive/IR_AssnDatasets_Sem6/Assn1/CSE508_Winter2023_Dataset'
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as f:
            file_content = f.read()
            file_content = str(file_content, 'UTF-8')
            docs.append(file_content)

In [ ]:
docs

In [ ]:
sample_before= (docs[2], docs[3], docs[878], docs[332], docs[1200])
len(sample_before)

5

In [ ]:
def extract_content(file_content):
    start_title_index = file_content.find('<TITLE>') + len('<TITLE>')
    end_title_index = file_content.find('</TITLE>')
    title = file_content[start_title_index:end_title_index].strip()
    
    start_text_index = file_content.find('<TEXT>') + len('<TEXT>')
    end_text_index = file_content.find('</TEXT>')
    text = file_content[start_text_index:end_text_index].strip()
    
    combined_content = title + ' ' + text
    return combined_content


In [ ]:
extracted_content= []
for i in docs:
  file_content = i
  combined_content = extract_content(file_content)
  extracted_content.append(combined_content)
  

In [ ]:
len(extracted_content)

1400

In [ ]:
sample_after= (extracted_content[2], extracted_content[3], extracted_content[878], extracted_content[332], extracted_content[1200])

In [ ]:
for i in range(5):
  print("Sample " + str(i) + " before: " + sample_before[i])
  print()
  print("Sample " + str(i) + " after: " + sample_after[i])
  print(" ---------------------------------------------------------------------------------------------------------------------------------  -")

Q1) (ii) Pre processing


In [ ]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

def preprocess(text):
    # Lowercase the text
    text = text.lower()
    print()
    
    # Perform tokenization
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Remove punctuations
    punc_removed = [token.translate(str.maketrans("", "", string.punctuation)) for token in tokens]
    
    # Remove blank space tokens
    tokens = [token for token in punc_removed if token.strip()]
    
    return tokens

In [ ]:
type(extracted_content[0])

str

In [ ]:
# Example usage
preproData= []
for i in extracted_content:
  text= i
  tokens = preprocess(text)  
  preproData.append(tokens)


In [ ]:
preproData[0]

Q2) BOOLEAN QUERIES


In [ ]:
len(preproData)

1400

In [ ]:
list_freq = []
for i in preproData:
  word_freq = {}
  for word in i:
      if word in word_freq:
          word_freq[word] += 1
      else:
          word_freq[word] = 1
  list_freq.append(word_freq)

In [ ]:
list_freq[4]

In [ ]:
inverted_index = {}
count=0
for k in range(len(list_freq)):
  count+=1

  for i, doc in enumerate(list_freq[k]):

    
    for word in doc.split():
        if word in inverted_index:
            inverted_index[word].append(i)
        else:
            inverted_index[word] = [i]

# inverted_index = dict(sorted(inverted_index.items()))
print(count)

1400


In [ ]:
inverted_index['local']

In [ ]:
import pickle
with open("inverted_index.pickle", "wb") as handle:
    pickle.dump(inverted_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def intersection(list1, list2):
    """Helper function to perform the intersection of two lists."""
    i = j = 0
    result = []
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1
    return result

def union(list1, list2):
    """Helper function to perform the union of two lists."""
    i = j = 0
    result = []
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1
    result += list1[i:] + list2[j:]
    return result

def difference(list1, list2):
    """Helper function to perform the difference of two lists."""
    i = j = 0
    result = []
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            j += 1
    result += list1[i:]
    return result

**Pickle Code**

In [ ]:
import pickle

#Save function
def save(index, file_name):
  # Save the index to file
  with open(file_name, "wb") as f:
    pickle.dump(index, f)

#Load function
def load(file_name):
  # Load the positional index from the file
  with open(file_name, "rb") as f:
    loaded_index = pickle.load(f)
  return loaded_index


**Q3 i) Biword Inverted Index**

In [ ]:
def biword_inverted_index(preproData):
  #creating inverted index using dictionary with biwords as keys and doc Ids as values
  bw_invertindex = {}

  for id, li in enumerate(preproData):
    for i in range(len(li)-1):
      biwords = [li[i] + ' ' + li[i+1]]
      for bword in biwords:
          if bword in bw_invertindex:
            bw_invertindex[bword].add(id)
          else:
            bw_invertindex[bword] = {id}

    return bw_invertindex

In [ ]:
final_bw_invertindex = biword_inverted_index(preproData)
len(final_bw_invertindex)

75

In [ ]:
biword_inverted_index(preproData)


**Q3 ii**) Positional Index


In [ ]:
def sort_index(pos_index):
  myKeys = list(pos_index.keys())
  myKeys.sort()
  sorted_pos_index = {i: pos_index[i] for i in myKeys}
  return sorted_pos_index

In [ ]:
def positional_index(sample):

  # Initialize the file no.
  fileno = 0
 
  # Initialize the dictionary.
  pos_index = {}
  doc_id=-1
  for doc in sample:
    doc_id+=1
    init_pos=0
    for word in doc:
      init_pos+=1
      if word in pos_index:
        doc_list=pos_index[word]
        if doc_id in doc_list:
          pos_list=doc_list[doc_id]
          pos_list.append(init_pos)
          doc_list[doc_id]=pos_list

        else:
          pos_list=[]
          pos_list.append(init_pos)
          doc_list[doc_id]=pos_list 
        pos_index[word]= doc_list        


      else:
        doc_list={}
        pos_list=[]
        pos_list.append(init_pos)
        doc_list[doc_id]=pos_list 
        pos_index[word]= doc_list

     
  return sort_index(pos_index)  
      


In [ ]:
final_index=positional_index(preproData)
len(final_index)

8996

In [ ]:
#Save and Load the index
save( final_index, "pos_index.pkl")
loaded_index=load("pos_index.pkl")
print(len(loaded_index))

8996


**Q3 iii) Run Phrase Queries and Compare**

In [ ]:
def execute_query_positional(c1):
  return


In [ ]:
t=int(input())
for i in range(t):
  c1=input()
  tokens=preprocess(c1)
  #execute_query_bigram(c1)
  
  #execute_query_positional(c1)



In [ ]:
def execute_query_bigram():
  